In [2]:
import numpy as np
from ipywidgets import widgets, Layout
import os
from pint import UnitRegistry

from canny_widgets import ConfigWidget
from canny_model import Data

import netCDF4

from scipy import (special, signal, ndimage)
from scipy.ndimage import gaussian_filter

import bokeh
from bokeh.plotting import (figure, show, output_notebook)
from bokeh.models import (LinearColorMapper, Ticker, ColorBar)
from bokeh.io import (push_notebook)

import os
from copy import (copy)

output_notebook()

Loading BokehJS ...

In [8]:
"data/cmip5/tas_Amon_MPI-ESM-LR_rcp85_r1i1p1_200601-210012.nc"
config = ConfigWidget()
config

In [4]:
data = Data(config)

In [5]:
from ipywidgets import (interact)

def plot_cube(plot_data):
    n, m = plot_data.min(), plot_data.max()
    fig = figure(x_range=(0, plot_data.shape[1]), y_range=(0, plot_data.shape[2]), plot_width=768, plot_height=384)
    clm = LinearColorMapper(palette=bokeh.palettes.viridis(255), low=n, high=m)
    img = fig.image(image=[plot_data[0,:,:]], x=0, y=0, dw=plot_data.shape[1], dh=plot_data.shape[2], color_mapper = clm)

    def update(y):  # , m):
        t = (y - 2006) # * 12 + m
        img.data_source.data['image'] = [plot_data[t]]
        push_notebook()

    show(fig, notebook_handle=True)
    interact(update, y=(2006, 2006+plot_data.shape[0] - 1))

In [7]:
#plot_cube(data.selected_data)
plot_cube(1./data.sobel_data[3])